In [3]:
%%writefile ../src/MyBank.sol
pragma solidity ^0.6.0;

contract MyBank{
    address owner;
    uint balance;
    
    constructor() public {
        owner = msg.sender;
        balance = 0;
    }
    
    event Sent(address from, address to, uint amount );

    function forwardTo(address payable _receiver) public payable {
        balance -= msg.value;
        require(msg.sender == owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function widthdrawAll() public {
        balance -= address(this).balance;
        require(msg.sender == owner);
        msg.sender.transfer(address(this).balance);
    }

}

Overwriting ../src/MyBank.sol


In [4]:
!solc ../src/MyBank.sol --combined-json abi > ..\src\MyBankABI.json

In [11]:
!solc ../src/MyBank.sol --combined-json bin > ..\src\MyBankBIN.json

In [22]:
%%writefile ..\src\MyBankBIN.json
{"contracts":{"../src/MyBank.sol:MyBank":{"bin":"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006001819055506103cb806100686000396000f3fe608060405260043610610062576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe01461006757806327d8ad88146100995780633c459375146100dd578063b6b55f25146100f4575b600080fd5b34801561007357600080fd5b5061007c610122565b604051808381526020018281526020019250505060405180910390f35b6100db600480360360208110156100af57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610148565b005b3480156100e957600080fd5b506100f261029c565b005b6101206004803603602081101561010a57600080fd5b810190808035906020019092919050505061037e565b005b6000806001543073ffffffffffffffffffffffffffffffffffffffff1631915091509091565b346001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156101b357600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156101f9573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b3073ffffffffffffffffffffffffffffffffffffffff16316001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561031e57600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f1935050505015801561037b573d6000803e3d6000fd5b50565b803414151561038c57600080fd5b806001600082825401925050819055505056fea165627a7a72305820c7944d870cff3ec335a018c77a49664f6919c627948eff3fd1579c63d8695ca90029"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Overwriting ..\src\MyBankBIN.json


In [14]:
!type ..\src\MyBankBIN.json

{"contracts":{"../src/MyBank.sol:MyBank":{"bin":"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055506000600181905550610367806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad88146100765780633c459375146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610260565b005b6100fd600480360360208110156100e757600080fd5b8101908080359060200190929190505050610312565b005b60008060015447915091509091565b346001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610177576

In [23]:
%%writefile ../src/MyBankDeploy.js
var Web3=require('web3');
var _abiJson = require('./MyBankABI.json');
var _binJson = require('./MyBankBIN.json');

web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8447"));

contractName=Object.keys(_abiJson.contracts); // reading ['src//Timer.sol:Timer'];
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin=_binJson.contracts[contractName].bin;

var _contract = new web3.eth.Contract(_abiArray);

_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x5c46b33dc12cf6ebbb834310ef235730f6c8449e", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log("- Contract Address: "+newContractInstance.options.address) // instance with the new contract address
    });

Overwriting ../src/MyBankDeploy.js


In [25]:
!node ../src/MyBankDeploy.js

- contract name:  ../src/MyBank.sol:MyBank
- Contract Address: 0x94cB2FE10e499fcb7260F88Ef8C42Bca9e4549F8


In [46]:
%%writefile ../src/BankUse.js
var Web3=require('web3');
var _abiJson = require('./MyBankABI.json');
var _binJson = require('./MyBankBIN.json');

var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8445"));

contractName=Object.keys(_abiJson.contracts); 
console.log("- contract name: ", contractName[0]); 
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);  
_bin=_binJson.contracts[contractName].bin;
var bank = new web3.eth.Contract(_abiArray,"0x94cB2FE10e499fcb7260F88Ef8C42Bca9e4549F8");


async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log(accounts);
    
    const balance0Before = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 before: " + balance0Before);
    const balance1Before = await web3.eth.getBalance(accounts[1]);
    console.log("Balance1 before: " + balance1Before);
    
    await bank.methods.deposit(11111)
    .send({from:accounts[0],gas:100000, value:11111})
    
    const balance0After = await web3.eth.getBalance(accounts[0]);
    console.log("Balance0 after: " + balance0After);
    const balance1After = await web3.eth.getBalance(accounts[1]);
    console.log("Balance1 after: " + balance1After);
}
doIt();

Overwriting ../src/BankUse.js


In [47]:
!node ../src/BankUse.js

- contract name:  ../src/MyBank.sol:MyBank
[
  '0x5C46B33Dc12cF6ebbb834310EF235730f6C8449e',
  '0x707C2942c4269BcaF5377B926D9FFc02610FD62d'
]
Balance0 before: 8534999999999999933334
Balance1 before: 0
Balance0 after: 8564999999999999922223
Balance1 after: 0
